<a href="https://colab.research.google.com/github/eeyamazaki/soebe/blob/main/modelo_preditivo_soebe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [583]:
# Importando as bibliotecas

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import glob
import datetime

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
from IPython.display import SVG
from graphviz import Source
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC

In [584]:
# Lendo a base dos serviços de reposição da contratada SOEBE:
# Base de serviços retirada da transação ZSBPM020 para as unidades: 277 e 273.
# skiprows = 3, pois as 3 primeiras linhas sao cabecalho.

dfSoebe = pd.read_csv(r'/content/drive/MyDrive/Projetos/soebe/pendentes-soebe/reposicoes.txt', encoding='cp1252', sep='|',
                      skiprows=3, dtype=object)
dfSoebe.head()

,Unnamed: 0,Ordem,Tipo ordem,Centro,Operação,CenTrab,Txt.breve operação,Cód. Único,Oper.ZSCP,TSE Oper.,...,Quantidade,Extensão,MND,L.Corte/Su,Religação,Mt.Cúbico,Observação,StatSist,StatUsuár.,Unnamed: 54
0,----------------------------------------------...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,2206463343,ZSCP,0004,0050,277CRB04,REPOSIÇÃO DE ASFALTO INV,226463343,0050,733000,...,,"0,00",,,,,...,LIB,EXEC VALO,NaN
2,NaN,2207236897,ZSCP,0004,0040,273CRL07,SERVIÇO EXECUTADO ANTERIORMENTE,227236897,0040,904200,...,,"0,00",,,,,"Conforme vistoria em 06.08.22, servico executa...",LIB,NEXE VALO,NaN
3,NaN,2210680011,ZSCP,0004,0030,277CRB04,REPOSIÇÃO DE ASFALTO INV,2210680011,0030,733000,...,,"0,00",,,,,...,LIB,EXEC VALO,NaN
4,NaN,2212237908,ZSCP,0004,0050,273CRL10,REPOSIÇÃO DE CAPA ASFALTICA INV,2212237908,0050,745000,...,,"0,00",,,,,...,LIB,EXEC VALO,NaN


In [585]:
# Como o cabecalho possui um excesso de espacos, vamos e renomeá-los:

l1 = dfSoebe.columns
l1 = [x.strip(' ') for x in l1]

dfSoebe = dfSoebe.rename(columns=dict(zip(dfSoebe.columns, l1)))
dfSoebe.columns

Index(['Unnamed: 0', 'Ordem', 'Tipo ordem', 'Centro', 'Operação', 'CenTrab',
       'Txt.breve operação', 'Cód. Único', 'Oper.ZSCP', 'TSE Oper.', 'TSS',
       'Cont.ZSCP', 'Un. Adm.', 'M.Mão Obra', 'Ag.Tp.Serv', 'Resp.ZSCP',
       'DtDespacho', 'Dt.Início', 'Hr.Iniíio', 'Dt.FimExec', 'Hr.Fim',
       'Hr.Trab.', 'Acatamento', 'Hr.Acatam.', 'Dt.Agend.', 'Turno',
       'Dt.Compet.', 'Dt.Susp.', 'Hr.Susp.', 'Dt.Fim', 'Hr.Fim  .1',
       'H.Retirado', 'Instalado', 'Garantia', 'Dt.Exec.', 'Hr.Exec.', 'Peso',
       'Tipo Mat.', 'Pos. Rede', 'Diam.Rede', 'Diam.Ramal', 'Diam.Cavl.',
       'Comp.', 'Largura', 'Profund.', 'Quantidade', 'Extensão', 'MND',
       'L.Corte/Su', 'Religação', 'Mt.Cúbico', 'Observação', 'StatSist',
       'StatUsuár.', 'Unnamed: 54'],
      dtype='object')

In [586]:
# Removendo excesso de espaços das colunas:

for i in l1:
  dfSoebe[i] = dfSoebe[i].str.strip()

In [587]:
# Missing Values
# Vamos remover os missing para as linhas em que não há ordem de serviço:

dfSoebe = dfSoebe.dropna(subset=['Ordem'])
dfSoebe.head()

,Unnamed: 0,Ordem,Tipo ordem,Centro,Operação,CenTrab,Txt.breve operação,Cód. Único,Oper.ZSCP,TSE Oper.,...,Quantidade,Extensão,MND,L.Corte/Su,Religação,Mt.Cúbico,Observação,StatSist,StatUsuár.,Unnamed: 54
1,NaN,2206463343,ZSCP,0004,0050,277CRB04,REPOSIÇÃO DE ASFALTO INV,226463343,0050,733000,...,,"0,00",,,,,,LIB,EXEC VALO,NaN
2,NaN,2207236897,ZSCP,0004,0040,273CRL07,SERVIÇO EXECUTADO ANTERIORMENTE,227236897,0040,904200,...,,"0,00",,,,,"Conforme vistoria em 06.08.22, servico executa...",LIB,NEXE VALO,NaN
3,NaN,2210680011,ZSCP,0004,0030,277CRB04,REPOSIÇÃO DE ASFALTO INV,2210680011,0030,733000,...,,"0,00",,,,,,LIB,EXEC VALO,NaN
4,NaN,2212237908,ZSCP,0004,0050,273CRL10,REPOSIÇÃO DE CAPA ASFALTICA INV,2212237908,0050,745000,...,,"0,00",,,,,,LIB,EXEC VALO,NaN
5,NaN,2213943855,ZSCP,0004,0020,277CRL06,DUPLICIDADE,2213943855,0020,901500,...,,"0,00",,,,,problema sistema,LIB,NEXE,NaN


In [588]:
# Ficamos com as seguintes colunas no dataset de reposições:

dfSoebe.columns

Index(['Unnamed: 0', 'Ordem', 'Tipo ordem', 'Centro', 'Operação', 'CenTrab',
       'Txt.breve operação', 'Cód. Único', 'Oper.ZSCP', 'TSE Oper.', 'TSS',
       'Cont.ZSCP', 'Un. Adm.', 'M.Mão Obra', 'Ag.Tp.Serv', 'Resp.ZSCP',
       'DtDespacho', 'Dt.Início', 'Hr.Iniíio', 'Dt.FimExec', 'Hr.Fim',
       'Hr.Trab.', 'Acatamento', 'Hr.Acatam.', 'Dt.Agend.', 'Turno',
       'Dt.Compet.', 'Dt.Susp.', 'Hr.Susp.', 'Dt.Fim', 'Hr.Fim  .1',
       'H.Retirado', 'Instalado', 'Garantia', 'Dt.Exec.', 'Hr.Exec.', 'Peso',
       'Tipo Mat.', 'Pos. Rede', 'Diam.Rede', 'Diam.Ramal', 'Diam.Cavl.',
       'Comp.', 'Largura', 'Profund.', 'Quantidade', 'Extensão', 'MND',
       'L.Corte/Su', 'Religação', 'Mt.Cúbico', 'Observação', 'StatSist',
       'StatUsuár.', 'Unnamed: 54'],
      dtype='object')

In [589]:
# Vamos remover variáveis que não trazem valor para o nosso problema:

dfSoebe = dfSoebe.drop(['Unnamed: 0', 'Tipo ordem', 'Centro', 'CenTrab', 'Txt.breve operação',
              'Cód. Único', 'Oper.ZSCP', 'TSS', 'Cont.ZSCP', 'M.Mão Obra',
              'Ag.Tp.Serv', 'Resp.ZSCP', 'DtDespacho', 'Dt.Início', 'Hr.Iniíio',
              'Hr.Fim', 'Hr.Trab.', 'Acatamento', 'Hr.Acatam.',
              'Dt.Agend.', 'Turno', 'Dt.Compet.', 'Dt.Susp.', 'Hr.Susp.',
              'Dt.Fim', 'Hr.Fim  .1', 'H.Retirado', 'Instalado', 'Garantia',
              'Dt.Exec.', 'Hr.Exec.', 'Peso', 'Tipo Mat.', 'Pos. Rede',
              'Diam.Rede', 'Diam.Ramal', 'Diam.Cavl.', 'Quantidade', 'Extensão',
              'MND', 'L.Corte/Su', 'Religação', 'Mt.Cúbico', 'Observação',
              'Unnamed: 54'], axis=1)

dfSoebe.head()

,Ordem,Operação,TSE Oper.,Un. Adm.,Dt.FimExec,Comp.,Largura,Profund.,StatSist,StatUsuár.
1,2206463343,0050,733000,277,14.07.2022,"1,00","1,00","0,00",LIB,EXEC VALO
2,2207236897,0040,904200,273,06.08.2022,"0,00","0,00","0,00",LIB,NEXE VALO
3,2210680011,0030,733000,277,14.07.2022,"2,00","1,50","0,00",LIB,EXEC VALO
4,2212237908,0050,745000,273,13.09.2022,"1,00","1,00","3,00",LIB,EXEC VALO
5,2213943855,0020,901500,277,16.09.2022,"0,00","0,00","0,00",LIB,NEXE


In [590]:
# Lendo a base de servicos liberados no SAP para o contrato do Global:

# skiprows = 3, pois as 3 primeiras linhas sao cabecalho.

dfGlobal = pd.read_csv(r'/content/drive/MyDrive/Projetos/soebe/pendentes-global/global.txt', 
                 encoding='cp1252', on_bad_lines='skip', skiprows=3, sep='|', dtype=object)
dfGlobal.head()

,Unnamed: 0,Ordem,Tipo ordem,Centro,Operação,CenTrab,Txt.breve operação,Cód. Único,Oper.ZSCP,TSE Oper.,...,Quantidade,Extensão,MND,L.Corte/Su,Religação,Mt.Cúbico,Observação,StatSist,StatUsuár.,Unnamed: 54
0,----------------------------------------------...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,2110062824,ZSCP,0004,0010,277CTA06,NADA CONSTA NO LOCAL,2110062824,0010,903000,...,,"0,00",,,,,...,LIB,NEXE,NaN
2,NaN,2110063329,ZSCP,0004,0010,277CTA06,NADA CONSTA NO LOCAL,2110063329,0010,903000,...,,"0,00",,,,,...,LIB,NEXE,NaN
3,NaN,2110063740,ZSCP,0004,0010,277CTA06,NADA CONSTA NO LOCAL,2110063740,0010,903000,...,,"0,00",,,,,...,LIB,NEXE,NaN
4,NaN,2110064727,ZSCP,0004,0010,277CTA06,NADA CONSTA NO LOCAL,2110064727,0010,903000,...,,"0,00",,,,,...,LIB,NEXE,NaN


In [591]:
# Como o cabecalho possui um excesso de espaços, vamos removê-los e renomeá-los:

l1 = dfGlobal.columns
l1 = [x.strip(' ') for x in l1]

dfGlobal = dfGlobal.rename(columns=dict(zip(dfGlobal.columns, l1)))

dfGlobal.columns

Index(['Unnamed: 0', 'Ordem', 'Tipo ordem', 'Centro', 'Operação', 'CenTrab',
       'Txt.breve operação', 'Cód. Único', 'Oper.ZSCP', 'TSE Oper.', 'TSS',
       'Cont.ZSCP', 'Un. Adm.', 'M.Mão Obra', 'Ag.Tp.Serv', 'Resp.ZSCP',
       'DtDespacho', 'Dt.Início', 'Hr.Iniíio', 'Dt.FimExec', 'Hr.Fim',
       'Hr.Trab.', 'Acatamento', 'Hr.Acatam.', 'Dt.Agend.', 'Turno',
       'Dt.Compet.', 'Dt.Susp.', 'Hr.Susp.', 'Dt.Fim', 'Hr.Fim  .1',
       'H.Retirado', 'Instalado', 'Garantia', 'Dt.Exec.', 'Hr.Exec.', 'Peso',
       'Tipo Mat.', 'Pos. Rede', 'Diam.Rede', 'Diam.Ramal', 'Diam.Cavl.',
       'Comp.', 'Largura', 'Profund.', 'Quantidade', 'Extensão', 'MND',
       'L.Corte/Su', 'Religação', 'Mt.Cúbico', 'Observação', 'StatSist',
       'StatUsuár.', 'Unnamed: 54'],
      dtype='object')

In [592]:
# Removendo o excesso de espacos no dataframe:

for i in l1:
  dfGlobal[i] = dfGlobal[i].str.strip()

In [593]:
# Missing Values:
# Removendo missing values pela Ordem no dataframe.

dfGlobal = dfGlobal.dropna(subset=['Ordem'])
dfGlobal.head()

,Unnamed: 0,Ordem,Tipo ordem,Centro,Operação,CenTrab,Txt.breve operação,Cód. Único,Oper.ZSCP,TSE Oper.,...,Quantidade,Extensão,MND,L.Corte/Su,Religação,Mt.Cúbico,Observação,StatSist,StatUsuár.,Unnamed: 54
1,NaN,2110062824,ZSCP,0004,0010,277CTA06,NADA CONSTA NO LOCAL,2110062824,0010,903000,...,,"0,00",,,,,,LIB,NEXE,NaN
2,NaN,2110063329,ZSCP,0004,0010,277CTA06,NADA CONSTA NO LOCAL,2110063329,0010,903000,...,,"0,00",,,,,,LIB,NEXE,NaN
3,NaN,2110063740,ZSCP,0004,0010,277CTA06,NADA CONSTA NO LOCAL,2110063740,0010,903000,...,,"0,00",,,,,,LIB,NEXE,NaN
4,NaN,2110064727,ZSCP,0004,0010,277CTA06,NADA CONSTA NO LOCAL,2110064727,0010,903000,...,,"0,00",,,,,,LIB,NEXE,NaN
5,NaN,2110065051,ZSCP,0004,0010,277CTA06,NADA CONSTA NO LOCAL,2110065051,0010,903000,...,,"0,00",,,,,,LIB,NEXE,NaN


In [594]:
# Ficamos com as seguintes colunas:

dfGlobal.columns

Index(['Unnamed: 0', 'Ordem', 'Tipo ordem', 'Centro', 'Operação', 'CenTrab',
       'Txt.breve operação', 'Cód. Único', 'Oper.ZSCP', 'TSE Oper.', 'TSS',
       'Cont.ZSCP', 'Un. Adm.', 'M.Mão Obra', 'Ag.Tp.Serv', 'Resp.ZSCP',
       'DtDespacho', 'Dt.Início', 'Hr.Iniíio', 'Dt.FimExec', 'Hr.Fim',
       'Hr.Trab.', 'Acatamento', 'Hr.Acatam.', 'Dt.Agend.', 'Turno',
       'Dt.Compet.', 'Dt.Susp.', 'Hr.Susp.', 'Dt.Fim', 'Hr.Fim  .1',
       'H.Retirado', 'Instalado', 'Garantia', 'Dt.Exec.', 'Hr.Exec.', 'Peso',
       'Tipo Mat.', 'Pos. Rede', 'Diam.Rede', 'Diam.Ramal', 'Diam.Cavl.',
       'Comp.', 'Largura', 'Profund.', 'Quantidade', 'Extensão', 'MND',
       'L.Corte/Su', 'Religação', 'Mt.Cúbico', 'Observação', 'StatSist',
       'StatUsuár.', 'Unnamed: 54'],
      dtype='object')

In [595]:
# Removendo as variaveis desnecessarias:

dfGlobal = dfGlobal.drop(['Unnamed: 0', 'Tipo ordem', 'Centro', 'CenTrab',
       'Txt.breve operação', 'Cód. Único', 'Oper.ZSCP', 'TSS', 
       'Cont.ZSCP', 'Un. Adm.', 'M.Mão Obra', 'Ag.Tp.Serv', 'Resp.ZSCP',
       'DtDespacho', 'Dt.Início', 'Hr.Iniíio', 'Dt.FimExec', 'Hr.Fim',
       'Hr.Trab.', 'Acatamento', 'Hr.Acatam.', 'Dt.Agend.', 'Turno',
       'Dt.Compet.', 'Dt.Susp.', 'Hr.Susp.', 'Dt.Fim', 'Hr.Fim  .1',
       'H.Retirado', 'Instalado', 'Garantia', 'Dt.Exec.', 'Hr.Exec.', 'Peso',
       'Quantidade', 'Observação', 'Unnamed: 54', 'L.Corte/Su', 'Religação'], axis=1)

dfSoebe.head()

,Ordem,Operação,TSE Oper.,Un. Adm.,Dt.FimExec,Comp.,Largura,Profund.,StatSist,StatUsuár.
1,2206463343,0050,733000,277,14.07.2022,"1,00","1,00","0,00",LIB,EXEC VALO
2,2207236897,0040,904200,273,06.08.2022,"0,00","0,00","0,00",LIB,NEXE VALO
3,2210680011,0030,733000,277,14.07.2022,"2,00","1,50","0,00",LIB,EXEC VALO
4,2212237908,0050,745000,273,13.09.2022,"1,00","1,00","3,00",LIB,EXEC VALO
5,2213943855,0020,901500,277,16.09.2022,"0,00","0,00","0,00",LIB,NEXE


In [596]:
# Quantidade de Ordens duplicadas:

# Aqui a ordem pode duplicar porque para uma mesma ordem de serviço podemos ter etapas diferentes que foram executadas:

len(dfGlobal['Ordem'])-dfGlobal['Ordem'].nunique()

14789

In [597]:
# As etapas pendentes de valoração assumem o valor StatUsuario como NEXE (não executado) ou EXEC (executado).
# NEXE não se paga, logo o valor é 0.
# EXEC a etapa foi executada e deve receber algum valor de acordo com o item de preço e análise dos serviços correlatos (extensão, largura, comprimento e outros detalhes do serviço principal).

dfGlobal['StatUsuár.'].unique()

array(['NEXE', 'EXEC VALO', 'EXEC', 'NEXE VALO'], dtype=object)

In [598]:
# Fazendo a leitura de todos os arquivos de serviços pagos pelas reposições, desde o início do contrato:

# Aqui esperamos relacionar o valor pago para cada ordem de serviço da contratada de acordo com as suas caracteristicas (extensao, largura, comprimento, servico principal atrelado, etc.)

path = r'/content/drive/MyDrive/Projetos/soebe/servicos_pagos-soebe'
filenames = glob.glob(path + "/*.txt")

dfs = []
for filename in filenames:
  dfs.append(pd.read_csv(filename, dtype=object, encoding='cp1252', sep='|',
                      skiprows=8))

dfReposicoesPagas = pd.concat(dfs, ignore_index=True)
dfReposicoesPagas.head()

,Unnamed: 0,Un. Adm.,Frente,Ordem,Etapa,TSE,Descrição do TSE,Endereço,Município,Data Fim d,...,Endereço,Descrição do Motivo de Não Pagamento,Endereço,Descrição,Pedido,Descrição do TSE,Endereço,Descrição do Motivo,Descrição do TSE,Descrição
0,----------------------------------------------...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,277,04,2206463343,0050,733000,REPOSIÇÃO DE ASFALTO INV,RUA MIGUEL OSORIO DE ALMEIDA 67 C/1 VL RENATO ...,SÃO PAULO,14.07.2022,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,277,04,2210680011,0030,733000,REPOSIÇÃO DE ASFALTO INV,RUA JOAO RICARDO 40 C/1 JD PERI ...,SÃO PAULO,14.07.2022,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,277,04,2218010936,0050,733000,REPOSIÇÃO DE ASFALTO INV,RUA FRANCISCO RODRIGUES ALVES 187 VL BRUNA ...,SÃO PAULO,14.07.2022,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,273,,2218474146,0030,904200,SERVIÇO EXECUTADO ANTERIORMENTE,RUA PONTA GROSSA 40 P MANDAQUI ...,SÃO PAULO,22.06.2022,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [599]:
# Como o cabecalho possui um excesso de espaços, vamos removê-los e renomeá-los:

l1 = dfReposicoesPagas.columns
l1 = [x.strip(' ') for x in l1]

dfReposicoesPagas = dfReposicoesPagas.rename(columns=dict(zip(dfReposicoesPagas.columns, l1)))

dfReposicoesPagas.columns

Index(['Unnamed: 0', 'Un. Adm.', 'Frente', 'Ordem', 'Etapa', 'TSE',
       'Descrição do TSE', 'Endereço', 'Município', 'Data Fim d', 'Dt.Val.',
       'Pago?', 'Motivo de', 'Descrição do Motivo', 'Número de', 'Item de Pr',
       'Descrição do Item de Preço', 'UM', 'Valor Unit', 'Quantidade',
       'Compriment', 'Largura', 'Profundida', 'Valor Medi', 'Medição Pa',
       'Medição De', 'Medição De.1', 'Medição Es', 'Ajuste', 'Documento',
       'Documento.1', 'Documento.2', 'Documento.3', 'Pedido', 'Período',
       'Início', 'Fim', 'Unnamed: 37', 'Endereço',
       'Descrição do Motivo de Não Pagamento', 'Endereço', 'Descrição',
       'Pedido', 'Descrição do TSE', 'Endereço', 'Descrição do Motivo',
       'Descrição do TSE', 'Descrição'],
      dtype='object')

In [600]:
# Vamos remover as variáveis desnecessarias:

dfReposicoesPagas = dfReposicoesPagas.drop(['Unnamed: 0', 'Un. Adm.', 'Frente', 'TSE',
       'Descrição do TSE', 'Endereço', 'Município', 'Data Fim d', 'Dt.Val.',
       'Motivo de', 'Descrição do Motivo', 'Descrição do Item de Preço', 'Medição Pa',
       'Medição De', 'Medição De.1', 'Medição Es', 'Ajuste', 'Documento',
       'Documento.1', 'Documento.2', 'Documento.3', 'Pedido', 'Período',
       'Início', 'Fim', 'Unnamed: 37', 'Descrição do TSE', 'Endereço',
       'Descrição do Motivo', 'Endereço', 'Compriment', 'Largura', 'Profundida',  
       'Descrição do Motivo de Não Pagamento', 'Endereço', 'Descrição',
       'Pedido'], axis=1)

dfReposicoesPagas.head()

,Ordem,Etapa,Pago?,Número de,Item de Pr,UM,Valor Unit,Quantidade,Valor Medi
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2206463343,0050,S,453897,04040204,un,"108,56","1,00","108,56"
2,2210680011,0030,S,453897,04040204,un,"108,56","1,00","108,56"
3,2218010936,0050,S,453897,04040204,un,"108,56","1,00","108,56"
4,2218474146,0030,N,,,,"0,00","0,00","0,00"


In [601]:
# Removendo valores nulos para a Ordem:
# O dataset deve ter o valor pago por cada etapa da sua respectiva ordem de serviço.
# A mesma ordem de serviço pode ter mais de um item a ser valorado.
# Devemos ficar com um dataset contendo apenas as linhas de serviços e o valor pago.

mask = (dfReposicoesPagas['Ordem']=='nan')
dfReposicoesPagas = dfReposicoesPagas[~mask]
dfReposicoesPagas = dfReposicoesPagas.dropna(subset=['Ordem'])

dfReposicoesPagas.head()

,Ordem,Etapa,Pago?,Número de,Item de Pr,UM,Valor Unit,Quantidade,Valor Medi
1,2206463343,0050,S,453897,04040204,un,"108,56","1,00","108,56"
2,2210680011,0030,S,453897,04040204,un,"108,56","1,00","108,56"
3,2218010936,0050,S,453897,04040204,un,"108,56","1,00","108,56"
4,2218474146,0030,N,,,,"0,00","0,00","0,00"
5,2218852095,0030,S,453210,01040301,un,"216,61","1,00","216,61"


In [602]:
# Removendo excesso de espacos:

l1 = dfReposicoesPagas.columns

for i in l1:
  dfReposicoesPagas[i] = dfReposicoesPagas[i].str.strip()

In [603]:
# Ficamos com o seguinte dataset de caracteristicas dos servicos pendentes ou valorados da SOEBE:

dfSoebe.head()

,Ordem,Operação,TSE Oper.,Un. Adm.,Dt.FimExec,Comp.,Largura,Profund.,StatSist,StatUsuár.
1,2206463343,0050,733000,277,14.07.2022,"1,00","1,00","0,00",LIB,EXEC VALO
2,2207236897,0040,904200,273,06.08.2022,"0,00","0,00","0,00",LIB,NEXE VALO
3,2210680011,0030,733000,277,14.07.2022,"2,00","1,50","0,00",LIB,EXEC VALO
4,2212237908,0050,745000,273,13.09.2022,"1,00","1,00","3,00",LIB,EXEC VALO
5,2213943855,0020,901500,277,16.09.2022,"0,00","0,00","0,00",LIB,NEXE


In [604]:
# Ficamos com o seguinte dataset de caracteristicas dos servicos pendentes ou valorados do Global:

dfGlobal.head()

,Ordem,Operação,TSE Oper.,Tipo Mat.,Pos. Rede,Diam.Rede,Diam.Ramal,Diam.Cavl.,Comp.,Largura,Profund.,Extensão,MND,Mt.Cúbico,StatSist,StatUsuár.
1,2110062824,0010,903000,,,,,,"0,00","0,00","0,00","0,00",,,LIB,NEXE
2,2110063329,0010,903000,,,,,,"0,00","0,00","0,00","0,00",,,LIB,NEXE
3,2110063740,0010,903000,,,,,,"0,00","0,00","0,00","0,00",,,LIB,NEXE
4,2110064727,0010,903000,,,,,,"0,00","0,00","0,00","0,00",,,LIB,NEXE
5,2110065051,0010,903000,,,,,,"0,00","0,00","0,00","0,00",,,LIB,NEXE


In [605]:
# Ficamos com o seguinte dataset de valores pagos para as reposicoes da SOEBE:

dfReposicoesPagas.head()

,Ordem,Etapa,Pago?,Número de,Item de Pr,UM,Valor Unit,Quantidade,Valor Medi
1,2206463343,0050,S,453897,04040204,un,"108,56","1,00","108,56"
2,2210680011,0030,S,453897,04040204,un,"108,56","1,00","108,56"
3,2218010936,0050,S,453897,04040204,un,"108,56","1,00","108,56"
4,2218474146,0030,N,,,,"0,00","0,00","0,00"
5,2218852095,0030,S,453210,01040301,un,"216,61","1,00","216,61"


In [606]:
# Realizando a leitura da base de serviços pagos para o contrato do Global:

path = r'/content/drive/MyDrive/Projetos/soebe/servicos_pagos-global'
filenames = glob.glob(path + "/*.txt")

dfs = []
for filename in filenames:
  dfs.append(pd.read_csv(filename, dtype=object, encoding='cp1252', sep='|',
                      skiprows=8))

dfServicosPagosGlobal = pd.concat(dfs, ignore_index=True)
dfServicosPagosGlobal.head()

,Unnamed: 0,Un. Adm.,Frente,Ordem,Etapa,TSE,Descrição do TSE,Endereço,Município,Data Fim d,...,Início,Fim,Unnamed: 37,Endereço,Endereço,Descrição do Item de Preço,Endereço,Descrição do Motivo,Descrição do Motivo de Não Pagamento,Endereço
0,----------------------------------------------...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,273,02,2105325349,0010,134000,INSTALADO LACRE DIVERSOS,RUA CONDESSA SICILIANO 202 V SICILIANO ...,SÃO PAULO,19.02.2022,...,01.02.2022,28.02.2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,273,06,2105530070,0020,508000,SUBSTITUIDA LIGAÇÃO DE ESGOTO,RUA ALVES PORTO 63 V MEDEIROS ...,SÃO PAULO,20.10.2021,...,01.02.2022,28.02.2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,273,06,2105530070,0030,745000,REPOSIÇÃO DE CAPA ASFALTICA INV,RUA ALVES PORTO 63 V MEDEIROS ...,SÃO PAULO,20.10.2021,...,01.02.2022,28.02.2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,273,,2105530070,0040,502000,LIGAÇÃO DE ESGOTO ADICIONAL,RUA ALVES PORTO 63 V MEDEIROS ...,SÃO PAULO,20.10.2021,...,01.02.2022,28.02.2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [607]:
# Como o cabecalho possui um excesso de espaços, vamos removê-los e renomeá-los:

l1 = dfServicosPagosGlobal.columns
l1 = [x.strip(' ') for x in l1]

dfServicosPagosGlobal = dfServicosPagosGlobal.rename(columns=dict(zip(dfServicosPagosGlobal.columns, l1)))

dfServicosPagosGlobal.columns

Index(['Unnamed: 0', 'Un. Adm.', 'Frente', 'Ordem', 'Etapa', 'TSE',
       'Descrição do TSE', 'Endereço', 'Município', 'Data Fim d', 'Dt.Val.',
       'Pago?', 'Motivo de', 'Descrição do Motivo', 'Número de', 'Item de Pr',
       'Descrição do Item de Preço', 'UM', 'Valor Unit', 'Quantidade',
       'Compriment', 'Largura', 'Profundida', 'Valor Medi', 'Medição Pa',
       'Medição De', 'Medição De.1', 'Medição Es', 'Ajuste', 'Documento',
       'Documento .1', 'Documento .2', 'Documento .3', 'Pedido', 'Período',
       'Início', 'Fim', 'Unnamed: 37', 'Endereço', 'Endereço',
       'Descrição do Item de Preço', 'Endereço', 'Descrição do Motivo',
       'Descrição do Motivo de Não Pagamento', 'Endereço'],
      dtype='object')

In [608]:
for coluna in dfServicosPagosGlobal.columns:
  print('=========='*30)
  print(dfServicosPagosGlobal[coluna].head())

0    ----------------------------------------------...
1                                                  NaN
2                                                  NaN
3                                                  NaN
4                                                  NaN
Name: Unnamed: 0, dtype: object
0           NaN
1           273
2           273
3           273
4           273
Name: Un. Adm., dtype: object
0       NaN
1    02    
2    06    
3    06    
4          
Name: Frente, dtype: object
0           NaN
1    2105325349
2    2105530070
3    2105530070
4    2105530070
Name: Ordem, dtype: object
0      NaN
1    0010 
2    0020 
3    0030 
4    0040 
Name: Etapa, dtype: object
0       NaN
1    134000
2    508000
3    745000
4    502000
Name: TSE, dtype: object
0                                         NaN
1    INSTALADO LACRE DIVERSOS                
2    SUBSTITUIDA LIGAÇÃO DE ESGOTO           
3    REPOSIÇÃO DE CAPA ASFALTICA INV         
4    LIGAÇÃO DE ESGOTO ADICIONAL     

In [609]:
# Vamos remover as variáveis desnecessarias:

# Vamos deixar apenas a variavel do item de preço.
# Hipótese: a etapa principal ter sido paga em despesa ou investimento implica de alguma forma no valor a ser pago pela reposição.

dfServicosPagosGlobal = dfServicosPagosGlobal.drop(['Unnamed: 0', 'Un. Adm.', 'Frente', 'TSE',
       'Descrição do TSE', 'Endereço', 'Município', 'Data Fim d', 'Dt.Val.',
       'Pago?', 'Motivo de', 'Descrição do Motivo', 'Número de',
       'Descrição do Item de Preço', 'UM', 'Valor Unit', 'Quantidade',
       'Compriment', 'Largura', 'Profundida', 'Valor Medi', 'Medição Pa',
       'Medição De', 'Medição De.1', 'Medição Es', 'Ajuste', 'Documento',
       'Documento .1', 'Documento .2', 'Documento .3', 'Pedido', 'Período',
       'Início', 'Fim', 'Unnamed: 37', 'Endereço', 'Endereço',
       'Descrição do Item de Preço', 'Endereço', 'Descrição do Motivo',
       'Descrição do Motivo de Não Pagamento', 'Endereço'], axis=1)

dfServicosPagosGlobal.head()

,Ordem,Etapa,Item de Pr
0,NaN,NaN,NaN
1,2105325349,0010,02030205
2,2105530070,0020,06140102
3,2105530070,0030,06120603
4,2105530070,0040,


In [610]:
# Removendo os espaços em branco, restando apenas as linhas com ordens de serviços:

mask = (dfServicosPagosGlobal['Ordem']=='nan')
dfServicosPagosGlobal = dfServicosPagosGlobal[~mask]
dfServicosPagosGlobal = dfServicosPagosGlobal.dropna(subset=['Ordem'])

dfServicosPagosGlobal.head()

,Ordem,Etapa,Item de Pr
1,2105325349,0010,02030205
2,2105530070,0020,06140102
3,2105530070,0030,06120603
4,2105530070,0040,
5,2105530070,0050,


In [611]:
# Removendo excesso de espacos:

l1 = dfServicosPagosGlobal.columns

for i in l1:
  dfServicosPagosGlobal[i] = dfServicosPagosGlobal[i].str.strip()

In [612]:
# Ficamos com o seguinte dataframe:

dfServicosPagosGlobal.head()

,Ordem,Etapa,Item de Pr
1,2105325349,0010,02030205
2,2105530070,0020,06140102
3,2105530070,0030,06120603
4,2105530070,0040,
5,2105530070,0050,


In [613]:
# path = '/content/drive/MyDrive/Projetos/soebe/export/dfSoebe.csv'

# with open(path, 'w', encoding = 'utf-8-sig') as f:
#   dfSoebe.to_csv(f)

# path = '/content/drive/MyDrive/Projetos/soebe/export/dfGlobal.csv'

# with open(path, 'w', encoding = 'utf-8-sig') as f:
#   dfGlobal.to_csv(f)

# path = '/content/drive/MyDrive/Projetos/soebe/export/dfServPag.csv'

# with open(path, 'w', encoding = 'utf-8-sig') as f:
#   dfServPag.to_csv(f)

In [614]:
# Renomeando a coluna 'Etapa':
# Deixando a variavel com o mesmo nome do dataframe de servicos pendentes da SOEBE:

dfReposicoesPagas.rename(columns={'Etapa':'Operação'}, inplace=True)

In [615]:
# Para saber o valor pago por cada etapa de reposicao vamos realizar as seguintes etapas:

# Antes de agrupar a coluna Valor Medido, vamos transformar a variavel em tipo 'float':

# Temos valores no seguinte formato: 9.465,99

# Abordagem: 
# 1) vamos primeiro substituir o '.' por '-'
# 2) Substituir a ',' por '.'
# 3) Substituir o '-' por ''

dfReposicoesPagas['Valor Medi'] = dfReposicoesPagas['Valor Medi'].str.replace('.', '-')
dfReposicoesPagas['Valor Medi'] = dfReposicoesPagas['Valor Medi'].str.replace(',', '.')
dfReposicoesPagas['Valor Medi'] = dfReposicoesPagas['Valor Medi'].str.replace('-', '')

dfReposicoesPagas['Valor Medi'].unique()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  if sys.path[0] == '':


array(['108.56', '0.00', '216.61', '139.58', '298.64', '388.36', '478.08',
       '212.88', '109.02', '652.40', '62.80', '717.77', '657.53',
       '106.44', '328.12', '358.88', '538.32', '186.27', '26.61',
       '127.36', '447.32', '857.48', '580.62', '185.03', '341913.00',
       '612.04', '565.71', '133.05', '2065.78', '14801.72', '13171.18',
       '845.90', '622499.43', '155.09', '143.53', '239.49', '565.24',
       '128.15', '153.78', '11533.50', '891.92', '480.56', '2108.07',
       '23579.60', '749.68', '196.11', '27725.47', '875588.72', '345.94',
       '17111.12', '1700.33', '17434.69', '11673.88', '69.21', '425.77',
       '159.66', '532.21', '4993.71', '103.82', '256.30', '130.71',
       '27649.33', '5710.05', '17908.65', '384.45', '9465.99', '832.98',
       '888.20', '215.29', '426.74', '179.41', '615.20', '53.22',
       '1558.30', '1357.14', '638.65', '499.79', '1915.96', '7327.90',
       '13158.49', '6217.61', '422.90', '1011.20', '1037.81', '452.38',
       '1107.2

In [616]:
# Convertendo a variavel 'Valor Medi' em 'foat':

dfReposicoesPagas['Valor Medi'] = dfReposicoesPagas['Valor Medi'].astype(float)

In [617]:
# Agrupando por Ordem e Etapa e somando o valor pago por item:

# Essa operação é realizada para os casos em que há mais de um item de preço pago pela mesma ordem de serviço e etapa:

dfReposicoesPagas = dfReposicoesPagas.groupby(['Ordem', 'Operação'])['Valor Medi'].sum()

In [618]:
# Alterando o index para Ordem e Operacao para o dataframe da Soebe:

dfSoebe.set_index(['Ordem', 'Operação'], inplace=True)

In [619]:
# Merging nos dataframes para obtermos o valor pago para cada ordem:

dfSoebe = pd.merge(dfSoebe, dfReposicoesPagas, left_index=True, right_index=True, how='left')

In [620]:
# Resetando o index:

dfSoebe = dfSoebe.reset_index()

In [621]:
# Filtrando o dataframe do Global para retirar apenas as etapas 0010:
# Assumindo que as etapas principais assumem o valor de operacao = 0010 
# vamos deixar apenas as etapas principais atreladas às reposições, pois pode ser que elas contenham features
# que influenciem no valor a ser medido.

mask = dfGlobal['Operação']=='0010'
dfGlobal = dfGlobal[mask]
dfGlobal.head()

,Ordem,Operação,TSE Oper.,Tipo Mat.,Pos. Rede,Diam.Rede,Diam.Ramal,Diam.Cavl.,Comp.,Largura,Profund.,Extensão,MND,Mt.Cúbico,StatSist,StatUsuár.
1,2110062824,0010,903000,,,,,,"0,00","0,00","0,00","0,00",,,LIB,NEXE
2,2110063329,0010,903000,,,,,,"0,00","0,00","0,00","0,00",,,LIB,NEXE
3,2110063740,0010,903000,,,,,,"0,00","0,00","0,00","0,00",,,LIB,NEXE
4,2110064727,0010,903000,,,,,,"0,00","0,00","0,00","0,00",,,LIB,NEXE
5,2110065051,0010,903000,,,,,,"0,00","0,00","0,00","0,00",,,LIB,NEXE


In [622]:
# Merging dos dataframes pela Ordem:

df = pd.merge(dfSoebe, dfGlobal, left_on='Ordem', right_on='Ordem', how='left')

In [623]:
# Ficamos com as seguintes colunas:

df.columns

Index(['Ordem', 'Operação_x', 'TSE Oper._x', 'Un. Adm.', 'Dt.FimExec',
       'Comp._x', 'Largura_x', 'Profund._x', 'StatSist_x', 'StatUsuár._x',
       'Valor Medi', 'Operação_y', 'TSE Oper._y', 'Tipo Mat.', 'Pos. Rede',
       'Diam.Rede', 'Diam.Ramal', 'Diam.Cavl.', 'Comp._y', 'Largura_y',
       'Profund._y', 'Extensão', 'MND', 'Mt.Cúbico', 'StatSist_y',
       'StatUsuár._y'],
      dtype='object')

In [624]:
# No dataframe podemos observar a existencia de valores em branco, vamos trata-lo:
# Alterando valores em branco por '0':

df = df.replace(r'^s*$', float('NaN'), regex = True)
df['MND'] = df['MND'].fillna('Não')
df = df.fillna(0)

In [625]:
# Vamos transformar as variaveis em tipo 'float':

# Da mesma forma que transformamos os valores pagos, o mesmo pode ocorrer para comprimento, largura e profundidade.
# Abordagem: 
# 1) vamos primeiro substituir os '.' por '-'
# 2) Substituir a ',' por '.'
# 3) Substituir o '-' por ''

df['Comp._x'] = df['Comp._x'].str.replace('.', '-')
df['Comp._x'] = df['Comp._x'].str.replace(',', '.')
df['Comp._x'] = df['Comp._x'].str.replace('-', '')
df['Comp._x'] = df['Comp._x'].astype(float) 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  if __name__ == '__main__':


In [626]:
# O mesmo para a largura:

df['Largura_x'] = df['Largura_x'].str.replace('.', '-')
df['Largura_x'] = df['Largura_x'].str.replace(',', '.')
df['Largura_x'] = df['Largura_x'].str.replace('-', '')
df['Largura_x'] = df['Largura_x'].astype(float)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  This is separate from the ipykernel package so we can avoid doing imports until


In [627]:
# O mesmo para a profundidade:

df['Profund._x'] = df['Profund._x'].str.replace('.', '-')
df['Profund._x'] = df['Profund._x'].str.replace(',', '.')
df['Profund._x'] = df['Profund._x'].str.replace('-', '')
df['Profund._x'] = df['Profund._x'].astype(float)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  This is separate from the ipykernel package so we can avoid doing imports until


In [628]:
# O mesmo para o comprimento do serviço principal:

df['Comp._y'] = df['Comp._y'].str.replace('.', '-')
df['Comp._y'] = df['Comp._y'].str.replace(',', '.')
df['Comp._y'] = df['Comp._y'].str.replace('-', '')
df['Comp._y'] = df['Comp._y'].astype(float)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  This is separate from the ipykernel package so we can avoid doing imports until


In [629]:
# O mesmo para o comprimento do serviço principal:

df['Largura_y'] = df['Largura_y'].str.replace('.', '-')
df['Largura_y'] = df['Largura_y'].str.replace(',', '.')
df['Largura_y'] = df['Largura_y'].str.replace('-', '')
df['Largura_y'] = df['Largura_y'].astype(float)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  This is separate from the ipykernel package so we can avoid doing imports until


In [630]:
# O mesmo para a profundidade do serviço principal:

df['Profund._y'] = df['Profund._y'].str.replace('.', '-')
df['Profund._y'] = df['Profund._y'].str.replace(',', '.')
df['Profund._y'] = df['Profund._y'].str.replace('-', '')
df['Profund._y'] = df['Profund._y'].astype(float)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  This is separate from the ipykernel package so we can avoid doing imports until


In [631]:
# O mesmo para a extensão do serviço principal:

df['Extensão'] = df['Extensão'].str.replace('.', '-')
df['Extensão'] = df['Extensão'].str.replace(',', '.')
df['Extensão'] = df['Extensão'].str.replace('-', '')
df['Extensão'] = df['Extensão'].astype(float)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  This is separate from the ipykernel package so we can avoid doing imports until


In [632]:
# O mesmo para a metragem cúbica:

df['Mt.Cúbico'] = df['Mt.Cúbico'].str.replace('.', '-')
df['Mt.Cúbico'] = df['Mt.Cúbico'].str.replace(',', '.')
df['Mt.Cúbico'] = df['Mt.Cúbico'].str.replace('-', '')
df['Mt.Cúbico'] = df['Mt.Cúbico'].astype(float)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  This is separate from the ipykernel package so we can avoid doing imports until


In [633]:
# Avaliando missings:

df.isnull().sum()

Ordem              0
Operação_x         0
TSE Oper._x        0
Un. Adm.           0
Dt.FimExec         0
Comp._x            0
Largura_x          0
Profund._x         0
StatSist_x         0
StatUsuár._x       0
Valor Medi         0
Operação_y         0
TSE Oper._y        0
Tipo Mat.          0
Pos. Rede          0
Diam.Rede          0
Diam.Ramal         0
Diam.Cavl.         0
Comp._y         2591
Largura_y       2591
Profund._y      2591
Extensão        2591
MND                0
Mt.Cúbico       5273
StatSist_y         0
StatUsuár._y       0
dtype: int64

In [634]:
# Replacing null values for 0:

df['Extensão'] = df['Extensão'].fillna(0)
df['Profund._y'] = df['Profund._y'].fillna(0)
df['Largura_y'] = df['Largura_y'].fillna(0)
df['Comp._y'] = df['Mt.Cúbico'].fillna(0)
df['Mt.Cúbico'] = df['Mt.Cúbico'].fillna(0)
df.isnull().sum()

Ordem           0
Operação_x      0
TSE Oper._x     0
Un. Adm.        0
Dt.FimExec      0
Comp._x         0
Largura_x       0
Profund._x      0
StatSist_x      0
StatUsuár._x    0
Valor Medi      0
Operação_y      0
TSE Oper._y     0
Tipo Mat.       0
Pos. Rede       0
Diam.Rede       0
Diam.Ramal      0
Diam.Cavl.      0
Comp._y         0
Largura_y       0
Profund._y      0
Extensão        0
MND             0
Mt.Cúbico       0
StatSist_y      0
StatUsuár._y    0
dtype: int64

In [635]:
df.head()

,Ordem,Operação_x,TSE Oper._x,Un. Adm.,Dt.FimExec,Comp._x,Largura_x,Profund._x,StatSist_x,StatUsuár._x,...,Diam.Ramal,Diam.Cavl.,Comp._y,Largura_y,Profund._y,Extensão,MND,Mt.Cúbico,StatSist_y,StatUsuár._y
0,2206463343,0050,733000,277,14.07.2022,1.0,1.0,0.0,LIB,EXEC VALO,...,0,0,0.0,0.0,0.0,0.0,Não,0.0,LIB,EXEC VALO
1,2207236897,0040,904200,273,06.08.2022,0.0,0.0,0.0,LIB,NEXE VALO,...,0,0,0.0,0.0,0.0,0.0,Não,0.0,LIB,EXEC VALO
2,2210680011,0030,733000,277,14.07.2022,2.0,1.5,0.0,LIB,EXEC VALO,...,DN_20,0,0.0,0.0,0.6,0.0,Sim,0.0,LIB,EXEC VALO
3,2212237908,0050,745000,273,13.09.2022,1.0,1.0,3.0,LIB,EXEC VALO,...,0,0,0.0,0.0,0.0,0.0,Não,0.0,0,0
4,2213943855,0020,901500,277,16.09.2022,0.0,0.0,0.0,LIB,NEXE,...,0,0,0.0,0.0,0.0,0.0,Não,0.0,0,0


In [636]:
dfServicosPagosGlobal.head()

,Ordem,Etapa,Item de Pr
1,2105325349,0010,02030205
2,2105530070,0020,06140102
3,2105530070,0030,06120603
4,2105530070,0040,
5,2105530070,0050,


In [637]:
# Tratamento de valores nulos nos serviços pagos do global:
dfServicosPagosGlobal = dfServicosPagosGlobal.replace(r'^s*$', float('NaN'), regex = True)
dfServicosPagosGlobal.isnull().sum()

Ordem             0
Etapa             0
Item de Pr    41550
dtype: int64

In [638]:
# Tratamento de missing nos serviços pagos do global:
dfServicosPagosGlobal['Item de Pr'] = dfServicosPagosGlobal['Item de Pr'].fillna('00')
dfServicosPagosGlobal.isnull().sum()

Ordem         0
Etapa         0
Item de Pr    0
dtype: int64

In [639]:
dfServicosPagosGlobal.head()

,Ordem,Etapa,Item de Pr
1,2105325349,0010,02030205
2,2105530070,0020,06140102
3,2105530070,0030,06120603
4,2105530070,0040,00
5,2105530070,0050,00


In [640]:
# Vamos extrair os 2 primeiros caracteres do item de preço, esses valores definem se o serviço foi pago em despesa, investimento ou não pago (cesta ou não executado).

dfServicosPagosGlobal['Frente'] = dfServicosPagosGlobal['Item de Pr'].astype(str).str[:2]
dfServicosPagosGlobal['Frente'].value_counts()

00    41550
04    23388
01    10153
05     9179
06     7119
03     6584
02     6217
Name: Frente, dtype: int64

In [641]:
# Extraindo apenas os serviços cuja etapa = 0010:

mask = dfServicosPagosGlobal['Etapa']=='0010'
dfServicosPagosGlobal = dfServicosPagosGlobal[mask]

In [642]:
df.columns

Index(['Ordem', 'Operação_x', 'TSE Oper._x', 'Un. Adm.', 'Dt.FimExec',
       'Comp._x', 'Largura_x', 'Profund._x', 'StatSist_x', 'StatUsuár._x',
       'Valor Medi', 'Operação_y', 'TSE Oper._y', 'Tipo Mat.', 'Pos. Rede',
       'Diam.Rede', 'Diam.Ramal', 'Diam.Cavl.', 'Comp._y', 'Largura_y',
       'Profund._y', 'Extensão', 'MND', 'Mt.Cúbico', 'StatSist_y',
       'StatUsuár._y'],
      dtype='object')

In [643]:
df['StatUsuár._x'].value_counts()

EXEC VALO    3866
NEXE          689
EXEC          488
NEXE VALO     237
Name: StatUsuár._x, dtype: int64

In [644]:
# Contatenando os dataframes:

df = pd.merge(df, dfServicosPagosGlobal, left_on='Ordem', right_on='Ordem', how='left')

In [645]:
# Ficamos com as seguintes colunas:

df.columns

Index(['Ordem', 'Operação_x', 'TSE Oper._x', 'Un. Adm.', 'Dt.FimExec',
       'Comp._x', 'Largura_x', 'Profund._x', 'StatSist_x', 'StatUsuár._x',
       'Valor Medi', 'Operação_y', 'TSE Oper._y', 'Tipo Mat.', 'Pos. Rede',
       'Diam.Rede', 'Diam.Ramal', 'Diam.Cavl.', 'Comp._y', 'Largura_y',
       'Profund._y', 'Extensão', 'MND', 'Mt.Cúbico', 'StatSist_y',
       'StatUsuár._y', 'Etapa', 'Item de Pr', 'Frente'],
      dtype='object')

In [646]:
# Avaliando as variaveis do dataframe:

for coluna in df.columns:
  print('**********'*30)
  print(df[coluna].value_counts())

************************************************************************************************************************************************************************************************************************************************************************************************************
2224279400    8
2221495867    5
2219720504    5
2233157533    5
2233116991    5
             ..
2227624066    1
2227623547    1
2227623476    1
2227623121    1
2236714551    1
Name: Ordem, Length: 5215, dtype: int64
************************************************************************************************************************************************************************************************************************************************************************************************************
0040    2566
0030    1928
0050    1201
0010     686
0060     250
0020     199
0070      41
0080       5
0090       1
Name: Operação_x, dtype: int64
**********************************

In [647]:
# Ficamos com as seguintes colunas:

df.columns

Index(['Ordem', 'Operação_x', 'TSE Oper._x', 'Un. Adm.', 'Dt.FimExec',
       'Comp._x', 'Largura_x', 'Profund._x', 'StatSist_x', 'StatUsuár._x',
       'Valor Medi', 'Operação_y', 'TSE Oper._y', 'Tipo Mat.', 'Pos. Rede',
       'Diam.Rede', 'Diam.Ramal', 'Diam.Cavl.', 'Comp._y', 'Largura_y',
       'Profund._y', 'Extensão', 'MND', 'Mt.Cúbico', 'StatSist_y',
       'StatUsuár._y', 'Etapa', 'Item de Pr', 'Frente'],
      dtype='object')

In [648]:
# Tipos das variaveis:

df.dtypes

Ordem            object
Operação_x       object
TSE Oper._x      object
Un. Adm.         object
Dt.FimExec       object
Comp._x         float64
Largura_x       float64
Profund._x      float64
StatSist_x       object
StatUsuár._x     object
Valor Medi      float64
Operação_y       object
TSE Oper._y      object
Tipo Mat.        object
Pos. Rede        object
Diam.Rede        object
Diam.Ramal       object
Diam.Cavl.       object
Comp._y         float64
Largura_y       float64
Profund._y      float64
Extensão        float64
MND              object
Mt.Cúbico       float64
StatSist_y       object
StatUsuár._y     object
Etapa            object
Item de Pr       object
Frente           object
dtype: object

In [649]:
df.head()

,Ordem,Operação_x,TSE Oper._x,Un. Adm.,Dt.FimExec,Comp._x,Largura_x,Profund._x,StatSist_x,StatUsuár._x,...,Largura_y,Profund._y,Extensão,MND,Mt.Cúbico,StatSist_y,StatUsuár._y,Etapa,Item de Pr,Frente
0,2206463343,0050,733000,277,14.07.2022,1.0,1.0,0.0,LIB,EXEC VALO,...,0.0,0.0,0.0,Não,0.0,LIB,EXEC VALO,0010,04061001,04
1,2207236897,0040,904200,273,06.08.2022,0.0,0.0,0.0,LIB,NEXE VALO,...,0.0,0.0,0.0,Não,0.0,LIB,EXEC VALO,NaN,NaN,NaN
2,2210680011,0030,733000,277,14.07.2022,2.0,1.5,0.0,LIB,EXEC VALO,...,0.0,0.6,0.0,Sim,0.0,LIB,EXEC VALO,0010,04061001,04
3,2212237908,0050,745000,273,13.09.2022,1.0,1.0,3.0,LIB,EXEC VALO,...,0.0,0.0,0.0,Não,0.0,0,0,NaN,NaN,NaN
4,2213943855,0020,901500,277,16.09.2022,0.0,0.0,0.0,LIB,NEXE,...,0.0,0.0,0.0,Não,0.0,0,0,NaN,NaN,NaN


In [650]:
# Removendo as variaveis desnecessarias:

df.drop(['StatSist_x', 'Operação_y', 'StatSist_y', 'StatUsuár._y', 'Etapa', 'Item de Pr'], axis=1, inplace=True)
df.head()

,Ordem,Operação_x,TSE Oper._x,Un. Adm.,Dt.FimExec,Comp._x,Largura_x,Profund._x,StatUsuár._x,Valor Medi,...,Diam.Rede,Diam.Ramal,Diam.Cavl.,Comp._y,Largura_y,Profund._y,Extensão,MND,Mt.Cúbico,Frente
0,2206463343,0050,733000,277,14.07.2022,1.0,1.0,0.0,EXEC VALO,108.56,...,0,0,0,0.0,0.0,0.0,0.0,Não,0.0,04
1,2207236897,0040,904200,273,06.08.2022,0.0,0.0,0.0,NEXE VALO,0.00,...,0,0,0,0.0,0.0,0.0,0.0,Não,0.0,NaN
2,2210680011,0030,733000,277,14.07.2022,2.0,1.5,0.0,EXEC VALO,108.56,...,75,DN_20,0,0.0,0.0,0.6,0.0,Sim,0.0,04
3,2212237908,0050,745000,273,13.09.2022,1.0,1.0,3.0,EXEC VALO,0.00,...,0,0,0,0.0,0.0,0.0,0.0,Não,0.0,NaN
4,2213943855,0020,901500,277,16.09.2022,0.0,0.0,0.0,NEXE,0.00,...,0,0,0,0.0,0.0,0.0,0.0,Não,0.0,NaN


In [651]:
# Confirmando se há valores missing:

df.isnull().sum()

Ordem              0
Operação_x         0
TSE Oper._x        0
Un. Adm.           0
Dt.FimExec         0
Comp._x            0
Largura_x          0
Profund._x         0
StatUsuár._x       0
Valor Medi         0
TSE Oper._y        0
Tipo Mat.          0
Pos. Rede          0
Diam.Rede          0
Diam.Ramal         0
Diam.Cavl.         0
Comp._y            0
Largura_y          0
Profund._y         0
Extensão           0
MND                0
Mt.Cúbico          0
Frente          2346
dtype: int64

In [652]:
# Tratando os valores missing sobre os itens de preço:

df['Frente'] = df['Frente'].fillna(99)

In [653]:
df.head()

,Ordem,Operação_x,TSE Oper._x,Un. Adm.,Dt.FimExec,Comp._x,Largura_x,Profund._x,StatUsuár._x,Valor Medi,...,Diam.Rede,Diam.Ramal,Diam.Cavl.,Comp._y,Largura_y,Profund._y,Extensão,MND,Mt.Cúbico,Frente
0,2206463343,0050,733000,277,14.07.2022,1.0,1.0,0.0,EXEC VALO,108.56,...,0,0,0,0.0,0.0,0.0,0.0,Não,0.0,04
1,2207236897,0040,904200,273,06.08.2022,0.0,0.0,0.0,NEXE VALO,0.00,...,0,0,0,0.0,0.0,0.0,0.0,Não,0.0,99
2,2210680011,0030,733000,277,14.07.2022,2.0,1.5,0.0,EXEC VALO,108.56,...,75,DN_20,0,0.0,0.0,0.6,0.0,Sim,0.0,04
3,2212237908,0050,745000,273,13.09.2022,1.0,1.0,3.0,EXEC VALO,0.00,...,0,0,0,0.0,0.0,0.0,0.0,Não,0.0,99
4,2213943855,0020,901500,277,16.09.2022,0.0,0.0,0.0,NEXE,0.00,...,0,0,0,0.0,0.0,0.0,0.0,Não,0.0,99


In [654]:
df.columns

Index(['Ordem', 'Operação_x', 'TSE Oper._x', 'Un. Adm.', 'Dt.FimExec',
       'Comp._x', 'Largura_x', 'Profund._x', 'StatUsuár._x', 'Valor Medi',
       'TSE Oper._y', 'Tipo Mat.', 'Pos. Rede', 'Diam.Rede', 'Diam.Ramal',
       'Diam.Cavl.', 'Comp._y', 'Largura_y', 'Profund._y', 'Extensão', 'MND',
       'Mt.Cúbico', 'Frente'],
      dtype='object')

In [655]:
# A cardinalidade de cada variavel segue conforme a seguir:

df.nunique()

Ordem           5215
Operação_x         9
TSE Oper._x       19
Un. Adm.           2
Dt.FimExec       132
Comp._x          402
Largura_x        275
Profund._x        22
StatUsuár._x       4
Valor Medi       131
TSE Oper._y       86
Tipo Mat.          6
Pos. Rede          6
Diam.Rede         11
Diam.Ramal         6
Diam.Cavl.         4
Comp._y            7
Largura_y         24
Profund._y        57
Extensão          43
MND                2
Mt.Cúbico          7
Frente             8
dtype: int64

In [656]:
# Convertendo os tipos de variáveis:

df['Ordem'] = df['Ordem'].astype(str)
df['Operação_x'] = df['Operação_x'].astype(str)
df['TSE Oper._x'] = df['TSE Oper._x'].astype(str)
df['Un. Adm.'] = df['Un. Adm.'].astype(str)
df['StatUsuár._x'] = df['StatUsuár._x'].astype(str)
df['TSE Oper._y'] = df['TSE Oper._y'].astype(str)
df['Tipo Mat.'] = df['Tipo Mat.'].astype(str)
df['Pos. Rede'] = df['Pos. Rede'].astype(str)
df['Diam.Ramal'] = df['Diam.Ramal'].astype(str)
df['Diam.Cavl.'] = df['Diam.Cavl.'].astype(str)
df['MND'] = df['MND'].astype(str)
df['Frente'] = df['Frente'].astype(str)
df['Dt.FimExec'] = pd.to_datetime(df['Dt.FimExec'])

In [657]:
# Variaveis categóricas com cardinalidade menor ou igual a 10, utilizamos o get_dummies:

dum = pd.get_dummies(df,
                    columns=['TSE Oper._x', 'Un. Adm.', 'Tipo Mat.', 'Pos. Rede',
                             'Diam.Rede','Diam.Ramal', 'Diam.Cavl.', 'MND', 'Frente'],
                    drop_first=True,
                    prefix=['TSE', 'UnAdm', 'TipoMat', 'PosRede',
                            'DRede', 'DRamal', 'DCav', 'MND', 'Frente'],
                    prefix_sep='_')
dum.head()

,Ordem,Operação_x,Dt.FimExec,Comp._x,Largura_x,Profund._x,StatUsuár._x,Valor Medi,TSE Oper._y,Comp._y,...,DCav_DN_50,DCav_DN_80,MND_Sim,Frente_01,Frente_02,Frente_03,Frente_04,Frente_05,Frente_06,Frente_99
0,2206463343,0050,2022-07-14,1.0,1.0,0.0,EXEC VALO,108.56,286000,0.0,...,0,0,0,0,0,0,1,0,0,0
1,2207236897,0040,2022-06-08,0.0,0.0,0.0,NEXE VALO,0.00,764500,0.0,...,0,0,0,0,0,0,0,0,0,1
2,2210680011,0030,2022-07-14,2.0,1.5,0.0,EXEC VALO,108.56,286000,0.0,...,0,0,1,0,0,0,1,0,0,0
3,2212237908,0050,2022-09-13,1.0,1.0,3.0,EXEC VALO,0.00,0,0.0,...,0,0,0,0,0,0,0,0,0,1
4,2213943855,0020,2022-09-16,0.0,0.0,0.0,NEXE,0.00,0,0.0,...,0,0,0,0,0,0,0,0,0,1


In [658]:
#Removendo index das dummies:
dum.reset_index(drop=True, inplace=True)

In [659]:
# Reavaliando a cardinalidade

df.nunique()

Ordem           5215
Operação_x         9
TSE Oper._x       19
Un. Adm.           2
Dt.FimExec       132
Comp._x          402
Largura_x        275
Profund._x        22
StatUsuár._x       4
Valor Medi       131
TSE Oper._y       86
Tipo Mat.          6
Pos. Rede          6
Diam.Rede         11
Diam.Ramal         6
Diam.Cavl.         4
Comp._y            7
Largura_y         24
Profund._y        57
Extensão          43
MND                2
Mt.Cúbico          7
Frente             8
dtype: int64

In [660]:
# Variaveis com cardinalidade maior que 10, utilizamos o Label Encoder:
# BASE TESTE

from sklearn import preprocessing

le = preprocessing.LabelEncoder()

le_TSE = le.fit_transform(df['TSE Oper._y'])
le_TSE_df = pd.DataFrame(le_TSE, columns=['LE_TSE'])

df = pd.merge(dum, le_TSE_df, left_index=True, right_index=True)

In [670]:
# Vamos separar a base de treino baseado em 2 variáveis:

# Primeiro: o que está pendente de valoração, a varíavel 'StatUsuár._x' assume o valor EXEC ou NEXEC.

l1=['NEXE', 'EXEC']
mask = df['StatUsuár._x'].isin(l1)
teste = df[mask]
teste.head()

In [671]:
# Segundo: queremos predizer os valores dos serviços da medição vigente.
# Exemplo: se estamos fechando a medição de outubro, vamos considerar os serviços finalizados até outubro. 
# Aquilo que foi executado em novembro ou posteriormente ficará para a próxima medição.

# Dessa forma vamos determinar uma variável de fim de período para filtrarmos o dataframe de teste.

dtfim = '2022-11-01'

mask = teste['Dt.FimExec']<dtfim
teste = teste[mask]

In [677]:
# O restante do dataframe sera utilizado para treino do modelo:

l2 = ['EXEC VALO', 'NEXE VALO']
mask = df['StatUsuár._x'].isin(l2)

treino = df[mask]
treino.head()

,Ordem,Operação_x,Dt.FimExec,Comp._x,Largura_x,Profund._x,StatUsuár._x,Valor Medi,TSE Oper._y,Comp._y,...,DCav_DN_80,MND_Sim,Frente_01,Frente_02,Frente_03,Frente_04,Frente_05,Frente_06,Frente_99,LE_TSE
0,2206463343,0050,2022-07-14,1.0,1.0,0.0,EXEC VALO,108.56,286000,0.0,...,0,0,0,0,0,1,0,0,0,14
1,2207236897,0040,2022-06-08,0.0,0.0,0.0,NEXE VALO,0.00,764500,0.0,...,0,0,0,0,0,0,0,0,1,74
2,2210680011,0030,2022-07-14,2.0,1.5,0.0,EXEC VALO,108.56,286000,0.0,...,0,1,0,0,0,1,0,0,0,14
3,2212237908,0050,2022-09-13,1.0,1.0,3.0,EXEC VALO,0.00,0,0.0,...,0,0,0,0,0,0,0,0,1,0
5,2218010936,0050,2022-07-14,1.0,1.0,0.0,EXEC VALO,108.56,286000,0.0,...,0,0,0,0,0,1,0,0,0,14


In [ ]:
# Selecao de variaveis:
# BASE TESTE

explicativas_teste = teste[['Comp._x', 'Largura_x', 'Profund._x',
       'Comp._y', 'Largura_y', 'Profund._y', 'Extensão', 'Mt.Cúbico',
       'TSE_733000', 'TSE_743000', 'TSE_745000', 'TSE_901500',
       'TipoMat_CERAMICO', 'TipoMat_FF', 'TipoMat_PEAD', 'TipoMat_PVC',
       'PosRede_EI', 'PosRede_PO', 'PosRede_TA', 'PosRede_TO', 'DRede_100',
       'DRede_200', 'DRede_32', 'DRede_75', 'DRede_80', 'DRamal_DN_100',
       'DRamal_DN_20', 'DCav_DN_20', 'MND_Sim', 'LE_TSE']]

target_teste = teste[['Valor Medi']]

In [ ]:
# Selecao de variaveis:
#BASE TREINO

explicativas_treino = treino[['Comp._x', 'Largura_x', 'Profund._x',
       'Comp._y', 'Largura_y', 'Profund._y', 'Extensão', 'Mt.Cúbico',
       'TSE_733000', 'TSE_743000', 'TSE_745000', 'TSE_901500',
       'TipoMat_CERAMICO', 'TipoMat_FF', 'TipoMat_PEAD', 'TipoMat_PVC',
       'PosRede_EI', 'PosRede_PO', 'PosRede_TA', 'PosRede_TO', 'DRede_100',
       'DRede_200', 'DRede_32', 'DRede_75', 'DRede_80', 'DRamal_DN_100',
       'DRamal_DN_20', 'DCav_DN_20', 'MND_Sim', 'LE_TSE']]

target_treino = treino[['Valor Medi']]

In [ ]:
explicativas_treino.dtypes

Comp._x             float64
Largura_x           float64
Profund._x          float64
Comp._y             float64
Largura_y           float64
Profund._y          float64
Extensão            float64
Mt.Cúbico           float64
TSE_733000            uint8
TSE_743000            uint8
TSE_745000            uint8
TSE_901500            uint8
TipoMat_CERAMICO      uint8
TipoMat_FF            uint8
TipoMat_PEAD          uint8
TipoMat_PVC           uint8
PosRede_EI            uint8
PosRede_PO            uint8
PosRede_TA            uint8
PosRede_TO            uint8
DRede_100             uint8
DRede_200             uint8
DRede_32              uint8
DRede_75              uint8
DRede_80              uint8
DRamal_DN_100         uint8
DRamal_DN_20          uint8
DCav_DN_20            uint8
MND_Sim               uint8
LE_TSE                int64
dtype: object

In [ ]:
# Separando as bases de treino e teste:

x_train, x_test, y_train, y_test = train_test_split(explicativas_treino, target_treino, test_size=0.3, random_state=42)

In [ ]:
# Estanciando os modelos com as vairacoes de hiperparametros nos dicionarios:

from sklearn.tree import DecisionTreeRegressor

modelo_dt = DecisionTreeRegressor(random_state=42)

grid_dt = {
    "max_depth": [1, 3, 5, 8],
    "criterion":["squared_error", "friedman_mse", "absolute_error", "poisson"]
}

In [ ]:
#GridSearch CV da Regressão Logística:

grid_dt= GridSearchCV(modelo_dt,
                      grid_dt,
                      scoring='accuracy',
                      cv=10)

In [ ]:
#Treinando o modelo de Regressão Logística:
grid_dt.fit(x_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:774: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_scorer.py", line 221, in __call__
    sample_weight=sample_weight,
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_scorer.py", line 264, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py", line 211, in accuracy_score
    y_type, y_true, y_pred = _check_targets(y_true, y_pred)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py", line 104, in _check_targets
    raise ValueError("{0

GridSearchCV(cv=10, estimator=DecisionTreeRegressor(random_state=42),
             param_grid={'criterion': ['squared_error', 'friedman_mse',
                                       'absolute_error', 'poisson'],
                         'max_depth': [1, 3, 5, 8]},
             scoring='accuracy')

In [ ]:
grid_dt.predict(x_test)

array([  157.78985149,   157.78985149,   157.78985149,   157.78985149,
         157.78985149,   157.78985149,   157.78985149,   157.78985149,
         157.78985149,   157.78985149,   157.78985149,   157.78985149,
         157.78985149,   157.78985149,   157.78985149,   157.78985149,
         157.78985149,   157.78985149,   157.78985149,   157.78985149,
         157.78985149,   157.78985149,   157.78985149,   157.78985149,
         157.78985149,   157.78985149,   157.78985149,   157.78985149,
         157.78985149,   157.78985149,   157.78985149,   157.78985149,
         157.78985149,   157.78985149,   157.78985149,   157.78985149,
         157.78985149,   157.78985149,   157.78985149,   157.78985149,
         157.78985149,   157.78985149,   157.78985149,   157.78985149,
         157.78985149,   157.78985149,   157.78985149,   157.78985149,
         157.78985149,   157.78985149,   157.78985149,   157.78985149,
         157.78985149,   157.78985149,   157.78985149,   157.78985149,
      

In [ ]:
grid_dt.predict(explicativas_teste)

array([  157.78985149,   157.78985149,   157.78985149,   157.78985149,
         157.78985149,   157.78985149,   157.78985149,   157.78985149,
         157.78985149,   157.78985149,   157.78985149,   157.78985149,
         157.78985149,   157.78985149,   157.78985149,   157.78985149,
         157.78985149,   157.78985149,   157.78985149,   157.78985149,
       19594.67666667,   157.78985149,   157.78985149,   157.78985149,
         157.78985149,   157.78985149,   157.78985149,   157.78985149,
         157.78985149,   157.78985149,   157.78985149,   157.78985149,
         157.78985149,   157.78985149,   157.78985149,   157.78985149,
         157.78985149,   157.78985149,   157.78985149,   157.78985149,
         157.78985149,   157.78985149,   157.78985149,   157.78985149,
         157.78985149,   157.78985149,   157.78985149,   157.78985149,
         157.78985149,   157.78985149,   157.78985149,   157.78985149,
         157.78985149,   157.78985149,   157.78985149,   157.78985149,
      

In [ ]:
from sklearn.metrics import r2_score

predict_dtr = grid_dt.predict(x_test)
r2_dtr = r2_score(y_test, predict_dtr)
r2_dtr

-0.37533759263657385

In [ ]:
x_test

,Comp._x,Largura_x,Profund._x,TSE Oper._y,Comp._y,Largura_y,Profund._y,Extensão,Mt.Cúbico,TSE_733000,...,DRede_100,DRede_200,DRede_32,DRede_75,DRede_80,DRamal_DN_100,DRamal_DN_20,DCav_DN_20,MND_Sim,LE_TSE
1036,5.00,1.50,0.04,732000,0.0,0.0,0.0,0.0,0.0,0,...,0,0,0,0,0,0,0,0,0,58
1124,2.00,2.00,0.04,284500,0.0,0.0,1.0,0.0,0.0,0,...,1,0,0,0,0,0,1,0,1,10
997,1.20,1.00,0.00,416000,0.0,0.0,1.0,0.0,0.0,0,...,0,0,0,1,0,0,1,0,0,30
1316,1.20,1.00,0.04,284500,0.0,0.0,70.0,0.0,0.0,0,...,0,0,0,1,0,0,1,0,1,10
529,1.20,1.00,0.04,734000,0.0,0.0,0.0,0.0,0.0,0,...,0,0,0,0,0,0,0,0,0,60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
535,1.00,1.00,0.00,280000,0.0,0.0,1.0,0.0,0.0,1,...,1,0,0,0,0,0,1,0,1,6
135,3.60,6.70,0.04,732000,0.0,0.0,0.0,0.0,0.0,0,...,0,0,0,0,0,0,0,0,0,58
756,0.65,0.55,0.00,254000,0.0,0.0,1.0,0.0,0.0,1,...,0,0,0,1,0,0,1,1,1,2
561,1.30,0.60,0.00,734000,0.0,0.0,0.0,0.0,0.0,1,...,0,0,0,0,0,0,0,0,0,60


In [ ]:
explicativas_teste.head()

,Comp._x,Largura_x,Profund._x,TSE Oper._y,Comp._y,Largura_y,Profund._y,Extensão,Mt.Cúbico,TSE_733000,...,DRede_100,DRede_200,DRede_32,DRede_75,DRede_80,DRamal_DN_100,DRamal_DN_20,DCav_DN_20,MND_Sim,LE_TSE
0,1.0,1.0,0.0,280000,0.0,0.0,1.0,0.0,0.0,1,...,1,0,0,0,0,0,1,0,1,1
1,1.0,1.0,0.0,280000,0.0,0.0,1.0,0.0,0.0,1,...,1,0,0,0,0,0,1,0,1,1
2,1.0,1.0,0.0,280000,0.0,0.0,0.0,0.0,0.0,1,...,0,0,0,0,0,0,0,0,0,1
3,1.0,1.0,0.0,280000,0.0,0.0,0.0,0.0,0.0,1,...,0,0,0,0,0,0,0,0,0,1
4,1.0,1.0,0.0,280000,0.0,0.0,0.0,0.0,0.0,1,...,0,0,0,0,0,0,0,0,0,1


In [ ]:
grid_dt.best_params_

{'criterion': 'squared_error', 'max_depth': 1}

In [ ]:
# Estanciando os modelos com as vairacoes de hiperparametros nos dicionarios:

from sklearn.ensemble import RandomForestRegressor

modelo_rf = RandomForestRegressor(random_state=42)

grid_rf = {
    "max_depth": [1, 3, 5, 8],
    "criterion":["squared_error", "absolute_error", "poisson"]
}

In [ ]:
#GridSearch CV do Random Forest:

grid_rf= GridSearchCV(modelo_rf,
                      grid_rf,
                      scoring='accuracy',
                      cv=10)

In [ ]:
#Treinando o modelo de Random Forest:

grid_rf.fit(x_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:774: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_scorer.py", line 221, in __call__
    sample_weight=sample_weight,
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_scorer.py", line 264, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/usr/local/lib

GridSearchCV(cv=10, estimator=RandomForestRegressor(random_state=42),
             param_grid={'criterion': ['squared_error', 'absolute_error',
                                       'poisson'],
                         'max_depth': [1, 3, 5, 8]},
             scoring='accuracy')

In [ ]:
grid_rf.predict(x_test)

array([  158.34864031,   158.34864031,   158.34864031,   158.34864031,
         158.34864031,   158.34864031,   158.34864031,   158.34864031,
         158.34864031,   158.34864031,   158.34864031,   158.34864031,
         158.34864031,   158.34864031,   158.34864031,   158.34864031,
         158.34864031,   158.34864031,   158.34864031,   158.34864031,
         158.34864031,   158.34864031,   158.34864031,   158.34864031,
         158.34864031,   158.34864031,   158.34864031,   158.34864031,
         158.34864031,   158.34864031,   158.34864031,   158.34864031,
         158.34864031,   158.34864031,   158.34864031,   158.34864031,
         158.34864031,   158.34864031,   158.34864031,   158.34864031,
         158.34864031,   158.34864031,   158.34864031,   158.34864031,
         158.34864031,   158.34864031,   158.34864031,   158.34864031,
         158.34864031,   158.34864031,   158.34864031,   158.34864031,
         158.34864031,   158.34864031,   158.34864031,   158.34864031,
      

In [ ]:
grid_rf.predict(explicativas_teste)

array([  158.34864031,   158.34864031,   158.34864031,   158.34864031,
         158.34864031,   158.34864031,   158.34864031,   158.34864031,
         158.34864031,   158.34864031,   158.34864031,   158.34864031,
         158.34864031,   158.34864031,   158.34864031,   158.34864031,
         158.34864031,   158.34864031,   158.34864031,   158.34864031,
       16909.6586508 ,   158.34864031,   158.34864031,   158.34864031,
         158.34864031,   158.34864031,   158.34864031,   158.34864031,
         158.34864031,   158.34864031,   158.34864031,   158.34864031,
         158.34864031,   158.34864031,   158.34864031,   158.34864031,
         158.34864031,   158.34864031,   158.34864031,   158.34864031,
         158.34864031,   158.34864031,   158.34864031,   158.34864031,
         158.34864031,   158.34864031,   158.34864031,   158.34864031,
         158.34864031,   158.34864031,   158.34864031,   158.34864031,
         158.34864031,   158.34864031,   158.34864031,   158.34864031,
      

In [ ]:
from sklearn.metrics import r2_score

predict_rfr = grid_rf.predict(x_test)
r2_rfr = r2_score(y_test, predict_rfr)
r2_rfr

-0.1274265905083607

In [ ]:
# Estanciando os modelos com as vairacoes de hiperparametros nos dicionarios:

from sklearn.ensemble import GradientBoostingRegressor

modelo_gb = GradientBoostingRegressor(random_state=42)

grid_gb = {
    "max_depth": [1, 3, 5, 8],
    "criterion":["friedman_mse", "squared_error", "mse"]
}

In [ ]:
#GridSearch CV do Gradient Boosting Regressor:

grid_gb= GridSearchCV(modelo_gb,
                      grid_gb,
                      scoring='accuracy',
                      cv=10)

In [ ]:
# Treinando o modelo de Gradient Boosting:

grid_gb.fit(x_test, y_test)

Streaming output truncated to the last 5000 lines.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/tree/_classes.py:363: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/tree/_classes.py:363: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/tree/_classes.py:363: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/tree/_classes.py:363: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  FutureWarning,
/usr/local/lib/p

GridSearchCV(cv=10, estimator=GradientBoostingRegressor(random_state=42),
             param_grid={'criterion': ['friedman_mse', 'squared_error', 'mse'],
                         'max_depth': [1, 3, 5, 8]},
             scoring='accuracy')

In [ ]:
grid_gb.predict(x_test)

array([  945.84515548,    95.941311  ,    95.941311  ,    95.941311  ,
         120.08859379,   201.70974329,   302.47886626,   120.08859379,
         302.47886626,   201.70974329,   201.70974329,   120.08859379,
          95.941311  ,   201.70974329,    95.941311  ,    95.941311  ,
         120.08859379,    95.941311  ,   201.70974329,    95.941311  ,
          95.941311  ,  1051.61358777,   220.85771676,    95.941311  ,
          95.941311  ,    95.941311  ,    95.941311  ,   201.70974329,
          95.941311  ,   201.70974329,    95.941311  ,    95.941311  ,
         969.99243827,    95.941311  ,  1474.80678241,   225.85702609,
          95.941311  ,   201.70974329,    95.941311  ,    95.941311  ,
         225.85702609,    95.941311  ,    95.941311  ,    95.941311  ,
         225.85702609,   120.08859379,   225.85702609,   302.47886626,
         225.85702609,    95.941311  ,    95.941311  ,   120.08859379,
         196.71043397,   201.70974329,   120.08859379,    95.941311  ,
      

In [ ]:
grid_gb.predict(explicativas_teste)

array([ 201.70974329,  201.70974329,  201.70974329,  201.70974329,
        201.70974329,  201.70974329,  201.70974329,  201.70974329,
        201.70974329,   95.941311  ,   95.941311  ,  201.70974329,
        201.70974329,  201.70974329,  201.70974329,  201.70974329,
        201.70974329,  201.70974329,  201.70974329,  201.70974329,
        945.84515548, 1051.61358777,  201.70974329,  302.47886626,
         95.941311  ,   95.941311  ,   95.941311  ,  201.70974329,
        201.70974329,  201.70974329,  201.70974329,   95.941311  ,
         95.941311  ,  201.70974329,  201.70974329,  201.70974329,
         95.941311  ,   95.941311  ,   95.941311  ,   95.941311  ,
        201.70974329,   95.941311  ,   95.941311  ,   95.941311  ,
         95.941311  ,  201.70974329,  196.71043397,   95.941311  ,
        201.70974329,   95.941311  ,   95.941311  , 1051.61358777,
         95.941311  ,   95.941311  ,   95.941311  ,   95.941311  ,
         95.941311  ,   95.941311  ,   95.941311  ,   95.94131

In [ ]:
from sklearn.metrics import r2_score

predict_gb = grid_gb.predict(x_test)
r2_gb = r2_score(y_test, predict_gb)
r2_gb

0.49223656710738806

In [ ]:
# Estanciando os modelos com as vairacoes de hiperparametros nos dicionarios:

from sklearn.neighbors import KNeighborsRegressor

modelo_kn = KNeighborsRegressor()

grid_kn = {
    "algorithm": ["auto", "ball_tree", "kd_tree", "brute"],
    "weights":["uniform", "distance"]
}

In [ ]:
grid_kn = GridSearchCV(modelo_kn,
                         grid_kn,
                         cv=10,
                       scoring='accuracy')

In [ ]:
grid_kn.fit(x_test, y_test)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:774: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_scorer.py", line 221, in __call__
    sample_weight=sample_weight,
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_scorer.py", line 264, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py", line 211, in accuracy_score
    y_type, y_true, y_pred = _check_targets(y_true, y_pred)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py", line 104, in _check_targets
    raise ValueError("{0

GridSearchCV(cv=10, estimator=KNeighborsRegressor(),
             param_grid={'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
                         'weights': ['uniform', 'distance']},
             scoring='accuracy')

In [ ]:
grid_kn.predict(x_test)

array([[  86.644],
       [ 108.836],
       [   0.   ],
       [ 108.928],
       [ 261.732],
       [ 139.58 ],
       [ 624.97 ],
       [   0.   ],
       [ 619.844],
       [ 139.58 ],
       [ 139.58 ],
       [   0.   ],
       [   0.   ],
       [  21.804],
       [ 216.61 ],
       [  41.528],
       [   0.   ],
       [   0.   ],
       [ 127.264],
       [   0.   ],
       [  37.006],
       [ 209.432],
       [  43.608],
       [   0.   ],
       [  25.12 ],
       [   0.   ],
       [  21.804],
       [ 139.58 ],
       [ 139.58 ],
       [ 139.58 ],
       [   0.   ],
       [ 155.344],
       [3941.838],
       [   0.   ],
       [3082.072],
       [ 139.58 ],
       [  62.8  ],
       [   0.   ],
       [ 216.61 ],
       [  62.8  ],
       [ 227.3  ],
       [  20.764],
       [   0.   ],
       [   0.   ],
       [ 111.664],
       [ 261.732],
       [  43.608],
       [ 435.272],
       [ 139.58 ],
       [   0.   ],
       [ 108.56 ],
       [ 133.468],
       [ 656

In [ ]:
grid_kn.predict(explicativas_teste)

array([[ 139.58 ],
       [ 139.58 ],
       [ 139.58 ],
       [ 139.58 ],
       [ 139.58 ],
       [ 139.58 ],
       [ 139.58 ],
       [ 139.58 ],
       [ 139.58 ],
       [ 139.58 ],
       [ 139.58 ],
       [ 139.58 ],
       [ 139.58 ],
       [ 139.58 ],
       [ 139.58 ],
       [ 139.58 ],
       [ 139.58 ],
       [ 139.58 ],
       [ 139.58 ],
       [ 139.58 ],
       [3928.764],
       [ 603.182],
       [ 606.77 ],
       [ 602.154],
       [   0.   ],
       [ 209.432],
       [   0.   ],
       [ 209.432],
       [ 139.58 ],
       [ 139.58 ],
       [ 139.58 ],
       [   0.   ],
       [ 155.344],
       [  21.804],
       [ 139.58 ],
       [ 139.58 ],
       [   0.   ],
       [   0.   ],
       [   0.   ],
       [   0.   ],
       [ 139.58 ],
       [ 216.61 ],
       [ 356.062],
       [   0.   ],
       [   0.   ],
       [ 139.58 ],
       [   0.   ],
       [  87.124],
       [ 139.58 ],
       [ 209.432],
       [ 209.432],
       [ 155.344],
       [ 139

In [ ]:
from sklearn.metrics import r2_score

predict_knr = grid_kn.predict(x_test)
r2_knr = r2_score(y_test, predict_knr)
r2_knr

0.453472249350659